# Query Checking with the DECLARE Checker

This tutorial explains how to perform the query checking of a DECLARE constraint in a log and how to browse the results. We first import the classes `DeclareQueryChecker` and `DeclareResultsBrowser` and the log.

In [1]:
import sys
import os
import pathlib

SCRIPT_DIR = pathlib.Path("../../../", "src").resolve()
sys.path.append(os.path.dirname(SCRIPT_DIR))

from src.Declare4Py.ProcessModels.DeclareModel import DeclareModel
from src.Declare4Py.ProcessMiningTasks.QueryChecking.DeclareQueryChecker import DeclareQueryChecker
from src.Declare4Py.ProcessMiningTasks.QueryChecking.DeclareResultsBrowser import DeclareResultsBrowser
from src.Declare4Py.D4PyEventLog import D4PyEventLog

log_path = os.path.join("../../../", "tests", "test_logs","Sepsis Cases.xes.gz")
event_log = D4PyEventLog(case_name="case:concept:name")
event_log.parse_xes_log(log_path)

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

The `DeclareQueryChecker` class takes as input:
- the event log;
- the boolean parameter `consider_vacuity=true` that means that vacuously satisfied traces are considered as satisfied, violated otherwise.
- The query/setting parameters `template`, `activation`, `target`, `activation_condition`, `target_condition` and `time_condition` that set the parameters of the DECLARE constraints and the variables to ask. If a parameter is not set then it is treated as a variable to ask.
- The float `min_support` parameter sets the support to be satisfied in the log by the variable assignments.
- the boolean parameter `return_first` that if set to false it returns all the variables assignments that satisfy the support in the log. Otherwise, it returns only one variables assignment (the first) that satisfy the support in the log. This saves computational time when one is only interested in the existence of a variable assignment with a given support.

In the following, we compute the assignments for the target by setting `template_str='Chain Response'`, `activation='IV Antibiotics'`, `act_cond='A.org:group is A'` and `min_support=0.2`:

After this setting, the method `run` of the `DeclareQueryChecker` class will perform the query checking.

In [2]:
query_checker = DeclareQueryChecker(log=event_log, consider_vacuity=False, template='Chain Response', activation='IV Antibiotics', activation_condition='A.org:group is A', min_support=0.2, return_first=False)
query_check_res: DeclareResultsBrowser = query_checker.run()

The query checking results can be filtered with the `filter_query_checking` method whose parameter `queries` takes as input a list of variables to ask. In the above example, we set the template and activation and ask for the targets. The output of `filter_query_checking` is a Pandas dataframe.

In [3]:
query_check_res.filter_query_checking(queries=['template', 'activation', 'target', 'activation_condition'])

,template,activation,target,activation_condition
0,Chain Response,ER Registration,ER Triage,A.org:group is A
1,Chain Response,ER Sepsis Triage,Leucocytes,A.org:group is A
2,Chain Response,ER Sepsis Triage,IV Liquid,A.org:group is A
3,Chain Response,IV Liquid,IV Antibiotics,A.org:group is A
4,Chain Response,IV Antibiotics,Admission NC,A.org:group is A


In the following example we compute the assignments both for the activation and the target by setting `template_str='Response'` and `min_support=0.8`:

In [4]:
query_checker = DeclareQueryChecker(log=event_log, consider_vacuity=False, template='Response', min_support=0.8, return_first=False)
query_check_res: DeclareResultsBrowser = query_checker.run()
    
query_check_res.filter_query_checking(queries=['template', 'activation', 'target'])

,template,activation,target
0,Response,ER Registration,Leucocytes
1,Response,ER Registration,CRP
2,Response,ER Registration,LacticAcid
3,Response,ER Registration,ER Triage
4,Response,ER Registration,ER Sepsis Triage
5,Response,ER Triage,Leucocytes
6,Response,ER Triage,CRP
7,Response,ER Triage,ER Sepsis Triage
8,Response,ER Sepsis Triage,Leucocytes
9,Response,ER Sepsis Triage,CRP


As last example, we set the activation and the target and ask for the template:

In [5]:
query_checker = DeclareQueryChecker(log=event_log, consider_vacuity=False, activation='ER Registration', target='CRP', min_support=0.2, return_first=False)
query_check_res: DeclareResultsBrowser = query_checker.run()

query_check_res.filter_query_checking(queries=['template', 'activation', 'target'])

,template,activation,target
0,Choice,ER Registration,CRP
1,Responded Existence,ER Registration,CRP
2,Response,ER Registration,CRP
3,Alternate Response,ER Registration,CRP
4,Precedence,ER Registration,CRP
5,Alternate Precedence,ER Registration,CRP
6,Not Chain Response,ER Registration,CRP
7,Not Chain Precedence,ER Registration,CRP
